In [15]:
import csv
import glob
import pandas as pd
from Bio import Phylo
from Bio import SeqIO
import ete3
from ete3 import Tree
import os.path as path
from datetime import date


### First we need to get the information from a "Settings_File". There we have all the paths of all the files we are going to need. 
Settings_File = input()
File_paths=[]
if path.isfile(Settings_File) == True:
    with open (Settings_File, 'r') as file:
        for line in file.readlines():
            File_paths.append(line.rstrip().split(':')[1])
        file.close()
       
    #Name of the reference specie we are going to use
    Specie_Name = File_paths[1]
    Reference_Specie = File_paths[2]
    #File called Duplications.tsv
    Duplications_File = File_paths[3]+'/Gene_Duplication_Events/Duplications.tsv'
    #Species labeled and non labeled trees
    Species_Tree = File_paths[3]+'/Species_Tree/SpeciesTree_rooted_node_labels.txt'
    Non_labeled_tree = File_paths[3]+'/Species_Tree/SpeciesTree_rooted.txt'
    #Folder containing all the files of the Orthologues of the reference specie
    Orthologues_Folder = File_paths[3]+'/Orthologues/Orthologues_' + Reference_Specie + '/' + Reference_Specie + '*.tsv'
    #All the proteins of the reference specie used
    Proteins_Fasta = File_paths[4]
    #Files containing the orthogroups (Orthogroups.tsv) and the orthogroups of the Unassigned genes(Orthogroups_UnassignedGenes.tsv)
    Orthogroups_File = File_paths[3] + '/Orthogroups/Orthogroups.tsv'
    Orthogroup_UnassignedGenes = File_paths[3] + '/Orthogroups/Orthogroups_UnassignedGenes.tsv'
    Output = File_paths[5]
    Support = float(File_paths[6])
    Added = ''
    for i in range(0,len(Reference_Specie)):
        if i >= len(Specie_Name):
            Added += Reference_Specie[i]
else:
    Configurations = ['If you are not sure about how to use this settings.txt file: please read the README.md','Specie_Name:','Reference_Specie:','OrthoFinder_path:','Reference_Specie_fasta_path:','Output:','Support:']
    with open ('OrthoEvolution_settings.txt','w') as file:
        file.writelines("%s\n" % i for i in Configurations)
    print('settings.txt file created')


OrthoEvolution_settings.txt


In [99]:
#-----------------------------------------------------------------------------------------------#
#DUPLICATIONS
#-----------------------------------------------------------------------------------------------#

#DataFrame showing all the duplications provided by OrthoFinder.
DUPLICATIONS_DF = pd.read_csv(Duplications_File, delimiter = "\t", header=0)

###############################################################################################################################################################
#Functions:
#With this function I am trying to get rid of all those duplicated genes in a Clade. It also renames the gene names in orden to have them all in the same format.
def extract_duplication_events (Clade_duplication_table):
    Total_dupl_genes = []
    Genes_1 = []
    Genes_2 = []
    for gene in Clade_duplication_table['Genes 1']:
        Genes_1.append(gene)
    for gene in Clade_duplication_table['Genes 2']:
        Genes_2.append(gene)
    Genes_1 = ','.join(Genes_1)
    Genes_1 = Genes_1.replace(' ','')
    Genes_2 = ','.join(Genes_2)
    Genes_2 = Genes_2.replace(' ','')
    Unique_Genes = Genes_1.split(',') + Genes_2.split(',')
    for gene in Unique_Genes:
        if Reference_Specie in gene:
            gene = gene.replace(Reference_Specie, '')
            gene = gene[1:]
            if gene not in Total_dupl_genes:
                Total_dupl_genes.append(gene)
    return Total_dupl_genes

#This function is used to obtain the species names from the oldest one to our Reference_Specie
def get_subtree (Reference_Specie, Tree):
    Clades = []
    for i in range(0,2):
        if Reference_Specie not in Tree[i].get_leaf_names():
            Clades.append(Tree[i].get_leaf_names())
        else:
            Subtree = Tree[i].get_children()
    return Clades, Subtree

#The following function is used to shorten the names of the species so it is easier to read them.
#Función para dejar los nombres de las especies sin adornos -> Cambiar según los nombres que le hayas puesto a las especies, se pueden añadir o quitar ifs y si se usa la misma estructura que tienen estos no debería haber problema.
def short_names (Species, Added):
    Clades_short = []
    for species in Species:
        if Added in species:
            Clades_short.append(species.replace(Added, ''))
    return Clades_short

In [100]:
#################################################################################################################################################################
#We are going to get the clades we are interested in (related to the reference specie) directly from the Species_Tree
S_tree = Phylo.read(Species_Tree, "newick")

Tree_Path = []
for clade in S_tree.trace('N0', Reference_Specie):
    c = str(clade)
    Tree_Path.append(c)
Tree_Path = list(reversed(Tree_Path))
Tree_Path.remove(Reference_Specie)
#Para poder buscar las Clades en el árbol necesitas que Tree_Path no contenga la especie referencia. Se vuelve a meter después.

#Names from the species tree -> With this we obtain a list of lists (if some species are agrupated they will be inside a list)
Tree = ete3.Tree(Non_labeled_tree)
Clades = []
for i in range(0,len(Tree_Path)):
    if i == 0:
        Clades += get_subtree(Reference_Specie, Tree.get_children())[0]
        Subtree = get_subtree(Reference_Specie, Tree.get_children())[1]
    else:
        Clades += get_subtree(Reference_Specie, Subtree)[0]
        Subtree = get_subtree(Reference_Specie, Subtree)[1]
Clades = list(reversed(Clades))
    
#It is useful though to have all the species names in just one list:        
Clades_Only = []
for element in Clades:
    if len(element) == 1:
        Clades_Only += element
    else:
        for clade in element:
            Clades_Only.append(clade)
Clades_Only = short_names(Clades_Only, Added)
    
Species_Path = []
for element in Clades:
    if len(element) == 1:
        Species_Path += short_names(element, Added)
            
    if len(element) > 1:
        Species_Path.append(element[1].replace(Added,'') + '_group')
            

#This code is used to now the node in wich the novo gene is originated
#Poner en el diccionario el easy_node
Clade_Path = {Specie_Name:Specie_Name}
for i in range(0,len(Tree_Path)):
    Clade_Path[Species_Path[i]] = Tree_Path[i]
    
#Para los siguientes pasos necesitas que la especie referencia esté en Tree_Path, para que salga ordenado tienes que girar la lista.
Tree_Path = list(reversed(Tree_Path))
Tree_Path.append(Reference_Specie)
Tree_Path = list(reversed(Tree_Path))


In [110]:
#I am obtaining the duplicated genes from each Clade (all repetitions inside each Clade are already eliminated)
Duplications_by_Clade_dic = {}
for i in range(0,len(Tree_Path)):
    One_Clade = []
    duplication_events_table = DUPLICATIONS_DF[(DUPLICATIONS_DF['Species Tree Node'] == Tree_Path[i]) & (DUPLICATIONS_DF['Support'] >= Support)]
    One_Clade += extract_duplication_events(duplication_events_table)
    Duplications_by_Clade_dic[Tree_Path[i]] = One_Clade



#Now I am comparing each Clade with each other in order to eliminate in between clades repetitions. As it is sorted from newer duplications to older the newest duplicated genes will be kept and the older one will be removed
All_Copies_dic = {}
for i in range(0,len(Tree_Path)):
    Copies_dic = {}
    for j in range(1, len(Tree_Path)):
        Copies_per_clade = []
        if Tree_Path[i] == Tree_Path[j]:
            continue
        elif j < i:
            continue
        else:
            for element in Duplications_by_Clade_dic.get(Tree_Path[j]):
                if element in Duplications_by_Clade_dic.get(Tree_Path[i]):
                    Copies_per_clade.append(element)
        Copies_dic[Tree_Path[j]] = Copies_per_clade
    All_Copies_dic[Tree_Path[i]] = Copies_dic

    
for i in range(0, len(Tree_Path)):
    for j in range(1, len(Tree_Path)):
        if Tree_Path[i] == Tree_Path[j]:
            continue
        elif j < i:
            continue
        else:
            for element in All_Copies_dic.get(Tree_Path[i]).get(Tree_Path[j]):  
                if element in Duplications_by_Clade_dic.get(Tree_Path[j]):
                    Duplications_by_Clade_dic.get(Tree_Path[j]).remove(element)
        

#Here we are getting the duplication events in each node. 
Duplication_events = []
for i in range(0,len(Tree_Path)):
    table = DUPLICATIONS_DF[(DUPLICATIONS_DF['Species Tree Node'] == Tree_Path[i]) & (DUPLICATIONS_DF['Support'] >= Support)]
    Duplication_events.append(len(table))


In [111]:
#It is important to be certain about the duplication events of the reference specie, that is why we are calculating that number in a diferent way:
Reference_Specie_DF = DUPLICATIONS_DF[(DUPLICATIONS_DF['Species Tree Node'] == Reference_Specie) & (DUPLICATIONS_DF['Support'] >= Support)] 

Orthogroups_copies = []
for Orthogroup in Reference_Specie_DF['Orthogroup']:
    Orthogroups_copies.append(Orthogroup)

Orthogroups = []    
for Orthogroup in Orthogroups_copies:
    if Orthogroup not in Orthogroups:
        Orthogroups.append(Orthogroup)

Total_genes_Orthogroup = []
for Orthogroup in Orthogroups:
    Genes_Orthogroup = []
    ORTHOGROUP_DF = Reference_Specie_DF[(Reference_Specie_DF['Orthogroup'] == Orthogroup)]
    for i in range(0,len(ORTHOGROUP_DF)):
        Genes_Orthogroup += ((ORTHOGROUP_DF.iloc[i]['Genes 1'].replace(' ','') + ',' + ORTHOGROUP_DF.iloc[i]['Genes 2'].replace(' ','')).split(','))
    Total_genes_Orthogroup.append(Genes_Orthogroup) 

In [112]:
Non_duplicated_genes_Orthogroup=[]
for element in Total_genes_Orthogroup:
    Non_duplicated_genes_Orthogroup.append(list(set(element)))

Duplication_Events_Ref_Spe = 0
for element in Non_duplicated_genes_Orthogroup:
    Duplication_Events_Ref_Spe += (len(element)-1)        

#In order to normalize the duplication events we need to now the branch length of each node.
Branch_Length = {}
for node in S_tree.find_clades(branch_length = True):
    Branch_Length[node.name] = node.branch_length

In [113]:
##########################################################################################################################################################################################    
Duplications_file = open(str(date.today()) + '-' + Output + "_Duplication_Analysis.tsv", "w")

writer = csv.writer(Duplications_file, delimiter = '\t')
writer.writerow(['Node','Genes'])
for key, value in Duplications_by_Clade_dic.items():
    for gene in value:
        writer.writerow([key.replace(Added, ''), gene])

Duplications_file.close()

In [114]:
##########################################################################################################################################################################################
#Creating a tsv file to save the overall information of the Gene Duplications Analysis
Duplications_Overall = open(str(date.today()) + '-' + Output + "_Duplication_Overall.tsv", "w")

writer = csv.writer(Duplications_Overall, delimiter = '\t')

counter = 0
writer.writerow(['Node','Specie','Duplications', 'Dupl_Events', 'Branch_length', 'Normalized_events'])
for key, value in Duplications_by_Clade_dic.items():
    if Specie_Name in key.replace(Added, ''):
        writer.writerow([key.replace(Added, ''), list(Clade_Path.keys())[counter], len(value),Duplication_Events_Ref_Spe, Branch_Length.get(key),(Duplication_Events_Ref_Spe)/(Branch_Length.get(key)*1000)])  
    if key.replace(Added, '') not in Branch_Length.keys() and Specie_Name not in key.replace(Added, ''):
        writer.writerow([key.replace(Added, ''), list(Clade_Path.keys())[counter],len(value),Duplication_events[counter],0,0])
    elif Specie_Name not in key:
        writer.writerow([key.replace(Added, ''), list(Clade_Path.keys())[counter],len(value),Duplication_events[counter], Branch_Length.get(key.replace(Added, '')),(Duplication_events[counter])/(Branch_Length.get(key.replace(Added, ''))*1000)])

    counter += 1

Duplications_Overall.close()

print("Analysis done, new file: " + str(date.today()) + '-' + Output + "_Duplication_Analysis.tsv")
print("Analysis done, new file: " + str(date.today()) + '-' + Output + "_Duplication_Overall.tsv")



Analysis done, new file: 2020-12-04-pruebas_Duplication_Analysis.tsv
Analysis done, new file: 2020-12-04-pruebas_Duplication_Overall.tsv


In [108]:
Specie_Name

'Drosophila_melanogaster'